# Benchmark of inference tools on 8CPU

Comparisson of different tools for model inference on 8 CPU.

|                     |                                         |
|---------------------|-------------------------------------------|
|**Hard ware**         | Intel Ice Lake with NVIDIA® Tesla® T4 with 4 CPUs and 16 Gb RAM.  |
|**Software Platform**| host |
|**Tools to compare**| Torch Script, Torch Trace, ONNX, Open Vino|

### Common

In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import timm
import torch as th
import typing as tp
import numpy as np
import cv2
import yaml
import json
import pandas as pd

from src.utils import get_batch, MAX_UINT8, benchmark


DATA_DIR = "./images"
BATCH_SIZES = [1,2,4,8,16]
PLATFORM = "16cpu"
WARMUP = 5
N_RUNS = 10
VERBOSE = False
N_PRINT = 5
DEVICE = "cpu"

In [ ]:
# batches = dict()
# for batch_sz in BATCH_SIZES:
#     batch = get_batch(DATA_DIR, batch_sz)
#     batches[batch_sz] = batch
#     del batch

In [ ]:
inference_records = []

### Torch Model

In [ ]:
from src.model.torch import ModelTorch

In [ ]:
records = []
with open("./config/torch.yaml") as fp:
    cfg_torch = yaml.safe_load(fp)

cfg_torch["device"] = DEVICE
model_torch = ModelTorch(cfg_torch)
for batch_sz in BATCH_SIZES:
    avg, stdev = benchmark(
        model=model_torch,
        input_shape=(batch_sz, 3, MAX_UINT8, MAX_UINT8),
        nwarmup=WARMUP,
        nruns=N_RUNS,
        print_step=N_PRINT,
        verbose=VERBOSE
    )
    records.append(
        {
            "time": avg,
            "platform": PLATFORM, 
            "batch_sz": batch_sz,
            "tool": "Torch"
        }
    )
inference_records.extend(records)
pd.DataFrame(records)


### TorchScript Model

In [ ]:
from src.model.torch_jit import ModelTorchJIT

tmp = th.jit.script(model_torch.model)
th.jit.save(tmp, "weights/model_scripted.th")

with open("./config/torch_scripted.yaml") as fp:
    cfg_scripted = yaml.safe_load(fp)
cfg_scripted["device"] = DEVICE
model_scripted = ModelTorchJIT(cfg_scripted)
records = []
for batch_sz in BATCH_SIZES:
    avg, stdev = benchmark(
        model=model_scripted,
        input_shape=(batch_sz, 3, MAX_UINT8, MAX_UINT8),
        nwarmup=WARMUP,
        nruns=N_RUNS,
        print_step=N_PRINT,
        verbose=VERBOSE
    )
    records.append(
        {
            "time": avg,
            "platform": PLATFORM, 
            "batch_sz": batch_sz,
            "tool": "TorchScript"
        }
    )
inference_records.extend(records)
pd.DataFrame(records)


### TorchTrace Model

In [ ]:
from src.model.torch_jit import ModelTorchJIT

In [ ]:
tmp = th.jit.trace(
    model_torch.model,
    th.rand(1, 3, MAX_UINT8, MAX_UINT8).to(th.float32)
)
th.jit.save(tmp, "weights/model_traced.th")

In [ ]:
with open("./config/torch_traced.yaml") as fp:
    cfg_traced = yaml.safe_load(fp)
cfg_traced["device"] = DEVICE
model_traced = ModelTorchJIT(cfg_traced)
records = []
for batch_sz in BATCH_SIZES:
    avg, _=benchmark(
        model=model_traced,
        input_shape=(batch_sz, 3, MAX_UINT8, MAX_UINT8),
        nwarmup=WARMUP,
        nruns=N_RUNS,
        print_step=N_PRINT,
        verbose=VERBOSE
    )
    records.append(
        {
            "time": avg,
            "platform": PLATFORM, 
            "batch_sz": batch_sz,
            "tool": "TorchTrace"
        }
    )
inference_records.extend(records)
pd.DataFrame(records)


### ONNX Model

In [ ]:
from src.model.onnx import ModelONNX

In [ ]:
th.onnx.export(
    model_torch.model,
    th.rand(1, 3, MAX_UINT8, MAX_UINT8).to(th.float32),
    "weights/model.onnx",
    verbose=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes = {
        'input': [0], 
        'output': [0]
    },
)

In [ ]:
with open("./config/onnx_cpu.yaml") as fp:
    cfg_onnx = yaml.safe_load(fp)
cfg_onnx["device"] = DEVICE
model_onnx = ModelONNX(cfg_onnx)
records = []
for batch_sz in BATCH_SIZES:
    avg, _ = benchmark(
        model=model_onnx,
        input_shape=(batch_sz, 3, MAX_UINT8, MAX_UINT8),
        nwarmup=WARMUP,
        nruns=N_RUNS,
        print_step=N_PRINT,
        verbose=VERBOSE
    )
    records.append(
        {
            "time": avg,
            "platform": PLATFORM, 
            "batch_sz": batch_sz,
            "tool": "ONNX"
        }
    )
inference_records.extend(records)
pd.DataFrame(records)


### OpenVino Model

In [ ]:
# bash command to convert ONNX -> OpenVino
!mo --input_model weights/model.onnx --output_dir weights/openvino

In [ ]:
from src.model.openvino import ModelOpenVino

In [ ]:
with open("./config/openvino.yaml") as fp:
    cfg_ov = yaml.safe_load(fp)
cfg_ov["device"] = DEVICE

model_ov = ModelOpenVino(cfg_ov)
records = []
for batch_sz in BATCH_SIZES:
    avg, _ = benchmark(
        model=model_ov,
        input_shape=(batch_sz, 3, MAX_UINT8, MAX_UINT8),
        nwarmup=WARMUP,
        nruns=N_RUNS,
        print_step=N_PRINT,
        verbose=VERBOSE
    )
    records.append(
        {
            "time": avg,
            "platform": PLATFORM, 
            "batch_sz": batch_sz,
            "tool": "OpenVino"
        }
    )
inference_records.extend(records)
pd.DataFrame(records)


In [ ]:
with open(
    "results/inference_results_16cpu.json", 
    "w",
    encoding="utf8"
) as fp:
    json.dump(fp=fp, obj=inference_records)

In [ ]:
pd.DataFrame(inference_records)